In [6]:
import os
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso, LassoCV, LinearRegression
from sklearn.model_selection import KFold
from utilities import cal_score

In [7]:
stack_idx = '05'
models = '1,3-7,9'
use_test_kfold = set([7])

### Read CV predictions and test

In [8]:
def parse_models(exp):
    exp_split = exp.split(',')
    idx_models = []
    for e in exp_split:
        if '-' in e:
            n0, n1 = e.split('-')
            idx_models.extend(list(range(int(n0), int(n1)+1, 1)))
        else:
            idx_models.append(int(e))
    return idx_models

In [9]:
idx_models = parse_models(models)

In [10]:
idx_models

[1, 3, 4, 5, 6, 7, 9]

In [11]:
files_in_output = [f for f in os.listdir('output/') if os.path.isfile('output/'+f)]

files_cv = {idx: [f for f in files_in_output if 'model-%02d-' % idx in f and 'cv' in f][0] for idx in idx_models}

files_test_one = {idx: [f for f in files_in_output if 'model-%02d-' % idx in f and 'test-one' in f][0] \
                  for idx in idx_models}
files_test_kf = {idx: [f for f in files_in_output if 'model-%02d-' % idx in f and 'test-kfold' in f][0] \
                 for idx in idx_models}

In [12]:
print(files_cv)
print(files_test_kf)
print(files_test_one)

{1: 'model-01-lgb-cv.csv', 3: 'model-03-lgb-feats-selection-cv.csv', 4: 'model-04-lgb-PCA-cv.csv', 5: 'model-05-lgb-wo-per-area-cv.csv', 6: 'model-06-lgb-lr0.001-cv.csv', 7: 'model-07-keras-embedding-cv.csv', 9: 'model-09-lgb-feats-selection-75-cv.csv'}
{1: 'model-01-lgb-test-kfold.csv', 3: 'model-03-lgb-feats-selection-test-kfold.csv', 4: 'model-04-lgb-PCA-test-kfold.csv', 5: 'model-05-lgb-wo-per-area-test-kfold.csv', 6: 'model-06-lgb-lr0.001-test-kfold.csv', 7: 'model-07-keras-embedding-test-kfold.csv', 9: 'model-09-lgb-feats-selection-75-test-kfold.csv'}
{1: 'model-01-lgb-test-one.csv', 3: 'model-03-lgb-feats-selection-test-one.csv', 4: 'model-04-lgb-PCA-test-one.csv', 5: 'model-05-lgb-wo-per-area-test-one.csv', 6: 'model-06-lgb-lr0.001-test-one.csv', 7: 'model-07-keras-embedding-test-one.csv', 9: 'model-09-lgb-feats-selection-75-test-one.csv'}


In [14]:
cv = None
test = None

for i, idx in enumerate(idx_models):
    f = files_cv[idx]
    df = pd.read_csv('output/'+f)
    
    if cv is None:
        cv = df[['building_id','total_price_predict']].copy()
    else:
        cv = pd.merge(cv, df[['building_id','total_price_predict']], on='building_id')
    
    cv = cv.rename(columns = {'total_price_predict':'pred_{}'.format(idx_models[i])})
    cv[f'log_pred_{idx_models[i]}'] = np.log1p(cv[f'pred_{idx_models[i]}'])

cv = pd.merge(cv, df[['building_id','total_price']], on='building_id')
cv['log_total_price'] = np.log1p(cv['total_price'])

for i, idx in enumerate(idx_models):
    f = files_test_kf[idx] if idx in use_test_kfold else files_test_one[idx]
    df = pd.read_csv('output/'+f)

    if test is None:
        test = df[['building_id','total_price']].copy()
    else:
        test = pd.merge(test, df[['building_id','total_price']], on='building_id')
        
    test = test.rename(columns = {'total_price':'pred_{}'.format(idx)})
    test[f'log_pred_{idx}'] = np.log1p(test[f'pred_{idx}'])

In [15]:
cv.head()

,building_id,pred_1,log_pred_1,pred_3,log_pred_3,pred_4,log_pred_4,pred_5,log_pred_5,pred_6,log_pred_6,pred_7,log_pred_7,pred_9,log_pred_9,total_price,log_total_price
0,jre1pJhcQj91Kdky,1.255350e+07,16.345510,1.255981e+07,16.346013,1.288381e+07,16.371482,1.282213e+07,16.366683,1.247484e+07,16.339224,12309220.0,16.325859,1.261616e+07,16.350489,14215011.0,16.469809
1,jcbuA8q3KPH9SzpS,9.117771e+06,16.025736,9.675385e+06,16.085096,7.095057e+06,15.774909,9.570172e+06,16.074162,9.175013e+06,16.031994,7909433.5,15.883567,9.376357e+06,16.053702,7642884.5,15.849286
2,EqWJpHmkXPyfHkB8,4.491672e+06,15.317736,4.511709e+06,15.322187,4.586149e+06,15.338551,4.687745e+06,15.360462,4.562418e+06,15.333364,4827711.0,15.389883,4.600447e+06,15.341664,4807053.0,15.385595
3,ihrMWGM8Nq99Uvu6,6.918995e+06,15.749781,6.878882e+06,15.743967,6.906435e+06,15.747964,6.981481e+06,15.758772,6.937283e+06,15.752421,7259765.5,15.797858,6.959559e+06,15.755627,7493026.5,15.829483
4,jRhwzoVMFLFzxAX1,2.750253e+06,14.827204,2.754684e+06,14.828814,2.725200e+06,14.818053,2.838090e+06,14.858642,2.766369e+06,14.833047,2949650.8,14.897198,2.727247e+06,14.818804,2846855.8,14.861726


In [16]:
X = cv[['log_pred_{}'.format(idx) for idx in idx_models]]
y = cv['log_total_price']

reg = LassoCV(alphas=[0]+list(np.logspace(-4, 3, 7)), max_iter=100000, tol=1e-6, n_jobs=-1)
reg.fit(X, y)

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:474: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  tol, rng, random, positive)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:474: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  tol, rng, random, positive)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:474: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  tol, rng, random, positive)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: C

LassoCV(alphas=[0, 0.0001, 0.0014677992676220704, 0.021544346900318846, 0.31622776601683794, 4.641588833612782, 68.12920690579622, 1000.0],
    copy_X=True, cv='warn', eps=0.001, fit_intercept=True, max_iter=100000,
    n_alphas=100, n_jobs=-1, normalize=False, positive=False,
    precompute='auto', random_state=None, selection='cyclic', tol=1e-06,
    verbose=False)

In [17]:
print(reg.alpha_)
print(reg.mse_path_)
print(reg.coef_, reg.intercept_)

0.0001
[[1.3539862  1.37718519 1.34124855]
 [1.3539862  1.37718519 1.34124855]
 [1.3539862  1.37718519 1.34124855]
 [0.11255819 0.11704416 0.10972506]
 [0.03595422 0.03810006 0.03677494]
 [0.0355604  0.0376592  0.0365321 ]
 [0.03555714 0.03765261 0.03653896]
 [0.03554657 0.03764356 0.03659121]]
[0.         0.26631984 0.01232962 0.27067851 0.2425683  0.15253216
 0.06170947] -0.0940669161948744


In [18]:
for a in [0]+list(np.logspace(-4, 3, 7)):
    reg_single = Lasso(alpha=a, max_iter=100000, tol=1e-6)
    reg_single.fit(X, y)
    print(reg_single.coef_, reg.intercept_)
    print(reg_single.score(X,y))

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:478: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


[-0.14541984  0.28117963  0.01282322  0.27252229  0.36025286  0.15254549
  0.07233539] -0.0940669161948744
0.973104868100518
[0.         0.26631984 0.01232962 0.27067851 0.2425683  0.15253216
 0.06170947] -0.0940669161948744
0.9731003429484877
[0.         0.2671007  0.00984    0.26702789 0.2449908  0.15286434
 0.06320916] -0.0940669161948744
0.9730990644674576
[0.         0.27713949 0.         0.21138691 0.26148732 0.15630291
 0.08269349] -0.0940669161948744
0.9728282062130117
[0.         0.30945399 0.         0.         0.         0.16798424
 0.28450805] -0.0940669161948744
0.9167204673966535
[0. 0. 0. 0. 0. 0. 0.] -0.0940669161948744
0.0
[0. 0. 0. 0. 0. 0. 0.] -0.0940669161948744
0.0
[0. 0. 0. 0. 0. 0. 0.] -0.0940669161948744
0.0


In [19]:
alphas = [0, 0.0001, 0.0002, 0.0005, 0.0008, 0.001, 0.002, 0.005, 0.008, 0.01, 0.02]
gsearch = {}

folds = KFold(n_splits=3, shuffle=True, random_state=1208)
for i_fold, (itrain, ival) in enumerate(folds.split(X)): # kfold
    print('==== Fold', i_fold+1, '====')
    
    # split train, val
    X_train = X.iloc[itrain]
    X_val = X.iloc[ival]
    y_train = y.iloc[itrain]
    y_val = y.iloc[ival]
    
    # random sample - grid search
    for a in alphas:
        if a == 0:
            reg_single = LinearRegression()
        else:
            reg_single = Lasso(alpha=a, max_iter=100000, tol=1e-6)
        reg_single.fit(X_train, y_train)
        
        y_pred = reg_single.predict(X_val)
        y_pred_final = np.expm1(y_pred)
        y_true_final = np.expm1(y_val)
        score = cal_score(y_true_final, y_pred_final)

        print('alpha, score:', a, score)
        gsearch[a] = gsearch.get(a,[]) + [score]

results = [[key, np.mean(value), value] for key, value, in gsearch.items()]
results.sort(key= lambda x: x[1], reverse=True)
for item in results:
    print(item)

==== Fold 1 ====
alpha, score: 0 5916.872608190963
alpha, score: 0.0001 5921.872585181543
alpha, score: 0.0002 5923.87258670467
alpha, score: 0.0005 5924.872590833311
alpha, score: 0.0008 5923.872594222182
alpha, score: 0.001 5925.872595987911
alpha, score: 0.002 5924.872600715948
alpha, score: 0.005 5923.8725656290935
alpha, score: 0.008 5920.872442529715
alpha, score: 0.01 5920.872307270934
alpha, score: 0.02 5871.871156451196
==== Fold 2 ====
alpha, score: 0 5886.873201436377
alpha, score: 0.0001 5880.873196686577
alpha, score: 0.0002 5880.873200649211
alpha, score: 0.0005 5881.873212132588
alpha, score: 0.0008 5880.873222855416
alpha, score: 0.001 5879.873229544782
alpha, score: 0.002 5876.873258812387
alpha, score: 0.005 5890.873295140916
alpha, score: 0.008 5896.873247334499
alpha, score: 0.01 5892.8731635482645
alpha, score: 0.02 5861.872261113058
==== Fold 3 ====
alpha, score: 0 5911.87740133396
alpha, score: 0.0001 5916.877419292948
alpha, score: 0.0002 5918.877421004957
alpha

In [20]:
alpha_set = results[0][0]
print(alpha_set)
if alpha_set == 0:
    reg = LinearRegression()
else:
    reg = Lasso(alpha=alpha_set, max_iter=1000000, tol=1e-6)
reg.fit(X, y)

0.008


Lasso(alpha=0.008, copy_X=True, fit_intercept=True, max_iter=1000000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=1e-06, warm_start=False)

In [21]:
print(reg.coef_, reg.intercept_)

[0.         0.27063769 0.         0.24943032 0.25503673 0.15434008
 0.07032579] 0.0047482405682934825


### Calculate cv score

In [22]:
cv_pred_final = np.zeros(X.shape[0])
for i, col in enumerate(X):
    cv_pred_final = cv_pred_final + X[col] * reg.coef_[i]
cv_pred_final = cv_pred_final + reg.intercept_
cv_pred_final = np.expm1(cv_pred_final)
cv_true_final = np.expm1(y)

In [23]:
pd.DataFrame({'a':cv_true_final,'b':cv_pred_final}).head()

,a,b
0,14215011.0,1.258027e+07
1,7642884.5,9.217113e+06
2,4807053.0,4.628002e+06
3,7493026.5,6.990691e+06
4,2846855.8,2.809537e+06


In [24]:
cal_score(cv_true_final, cv_pred_final)

5905.874376386471

### Compute submission

In [25]:
test_pred_final = pd.DataFrame({'building_id': test['building_id'], 'total_price': np.zeros(test.shape[0])})
for i, idx in enumerate(idx_models):
    test_pred_final['total_price'] = test_pred_final['total_price'] + test[f'log_pred_{idx}'] * reg.coef_[i]
test_pred_final['total_price'] = test_pred_final['total_price'] + reg.intercept_
test_pred_final['total_price'] = np.expm1(test_pred_final['total_price'])
test_pred_final['total_price'] = np.clip(test_pred_final['total_price'], 0, None)

test_pred_final.to_csv('output/stack_{}_{}.csv'.format(stack_idx, models), index=False)

In [ ]:
#a= pd.read_csv('output/model-03-lgb-feats-selection-cv.csv')
#b= pd.read_csv('output/model-03-lgb-feats-selection-test-one.csv')

In [ ]:
#a.rename(columns={'building_id':'id','total_price_predict':'target'}).to_csv('opt-pred3.csv',index=False)
#b.rename(columns={'building_id':'id','total_price':'target'}).to_csv('opt-test3.csv',index=False)